# Bitcoin Price Analysis Report

## Introduction

This report analyzes Bitcoin price data using various machine learning techniques to predict future price movements and identify market states.

## Data Overview

The analysis uses hourly Bitcoin price data including open, high, low, close prices and volume. Technical indicators were calculated to create features for the models.

## Prediction Models

Two models were trained to predict Bitcoin price movements:

1. **Random Forest Regressor**: Achieved an R² of 0.32, indicating it explains about 32% of the variance in price movements.
2. **Support Vector Regressor**: Showed negative R² performance, suggesting it may need further optimization.

## Market State Clustering

K-means clustering was used to identify distinct market states based on technical indicators. Four clusters were identified, representing different market conditions.

## Experiments

Several experiments were conducted:

1. **Data Augmentation**: Adding noise and synthetic samples to improve model robustness.
2. **Dimensionality Reduction**: Using PCA to reduce feature dimensionality while preserving information.

## Conclusions

The Random Forest model showed moderate predictive power for Bitcoin price movements. Clustering analysis revealed distinct market states with different characteristics. Future work could explore more sophisticated models and feature engineering techniques.